## Problem 1 : Transportation Problem

In [3]:
import sys
sys.setrecursionlimit(10000)

# Model (search problem)
class TransportationProblem(object):
    def __init__(self, N):
        # N = number of blocks
        self.N = N

    def startState(self):
        return 1

    def isEnd(self, state):
        return state == self.N

    def succAndCost(self, state):
        # return list of (action, newState, cost) triples
        result = []
        if state + 1 <= self.N:
            result.append(('walk', state + 1, 1))  # Walking cost: 1
        if state + 3 <= self.N:
            result.append(('run', state + 3, 1.5))  # Running cost: 1.5
        if state * 2 <= self.N:
            result.append(('tram', state * 2, 2))  # Tram cost: 2
        return result

In [5]:
def printSolution(solution):
    totalCost, history = solution
    print('totalCost: {}'.format(totalCost))
    for item in history:
        print(item)


### DFS

In [6]:
def depthFirstSearch(problem):
    best = {
        'cost': float('+inf'),
        'history': None
    }
    
    def recurse(state, history, totalCost):
        if problem.isEnd(state):
            if totalCost < best['cost']:
                best['cost'] = totalCost
                best['history'] = history
            return
        for action, newState, cost in problem.succAndCost(state):
            recurse(newState, history + [(action, newState, cost)], totalCost + cost)
    
    recurse(problem.startState(), history=[], totalCost=0)
    return (best['cost'], best['history'])


### BFS

In [7]:
from collections import deque
def breadthFirstSearch(problem):
    best = {
        'cost': float('+inf'),
        'history': None
    }
    
    queue = deque([(problem.startState(), [])])

    while queue:
        state, history = queue.popleft()

        if problem.isEnd(state) and len(history) > 0:
            totalCost = sum(cost for _, _, cost in history)
            if totalCost < best['cost']:
                best['cost'] = totalCost
                best['history'] = history
            continue

        for action, newState, cost in problem.succAndCost(state):
            queue.append((newState, history + [(action, newState, cost)]))

    return (best['cost'], best['history'])


### Dynamic Programming

In [20]:
def dynamicProgramming(problem):
    cache = {}  # state -> (futureCost(state), action)

    def futureCost(state):
        if problem.isEnd(state):
            return 0, None

        if state in cache:
            return cache[state]

        minCost = float('inf')
        bestAction = None

        for action, newState, cost in problem.succAndCost(state):
            future, _ = futureCost(newState)
            totalCost = cost + future

            if totalCost < minCost:
                minCost = totalCost
                bestAction = action

        cache[state] = minCost, bestAction
        return minCost, bestAction

    totalCost, bestAction = futureCost(problem.startState())

    # Track the sequence of actions and states with their respective costs
    history = []
    state = problem.startState()
    while state != problem.N and bestAction:
        for action, newState, cost in problem.succAndCost(state):
            if action == bestAction:
                history.append((action, newState, cost))
                state = newState
                break
        _, bestAction = futureCost(state)

    return totalCost, history


### UCS

In [9]:
import heapq

def uniformCostSearch(problem):
    heap = [(0, problem.startState(), [])]
    bestCost = float('inf')
    bestPath = []

    while heap:
        totalCost, state, history = heapq.heappop(heap)

        if problem.isEnd(state) and len(history) > 0:
            if totalCost < bestCost:
                bestCost = totalCost
                bestPath = history
            continue

        for action, newState, cost in problem.succAndCost(state):
            newCost = totalCost + cost
            heapq.heappush(heap, (newCost, newState, history + [(action, newState, cost)]))

    return (bestCost, bestPath)



In [28]:
# Problem
problem = TransportationProblem(N=23)

In [29]:
# Test DFS
printSolution(depthFirstSearch(problem))

totalCost: 8.0
('walk', 2, 1)
('run', 5, 1.5)
('tram', 10, 2)
('tram', 20, 2)
('run', 23, 1.5)


In [30]:
# Test BFS
printSolution(breadthFirstSearch(problem))

totalCost: 8.0
('walk', 2, 1)
('run', 5, 1.5)
('tram', 10, 2)
('tram', 20, 2)
('run', 23, 1.5)


In [31]:
# Test UCS
printSolution(uniformCostSearch(problem))


totalCost: 8.0
('run', 4, 1.5)
('run', 7, 1.5)
('run', 10, 1.5)
('tram', 20, 2)
('run', 23, 1.5)


In [32]:
# Test DP
printSolution(dynamicProgramming(problem))

totalCost: 8.0
('walk', 2, 1)
('run', 5, 1.5)
('tram', 10, 2)
('tram', 20, 2)
('run', 23, 1.5)
